In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join

In [24]:
# Define the file path
folder_path = "/media/kkondo/T7/dynus/tro_paper/time_allocation/csv/"
algorithms = ["faster_4", "faster_5", "faster_6", "liu"]

In [25]:
def get_header_initialized_dict(file, algo_folder_path):

    dict = {}

    # Print the file path
    file_path = algo_folder_path + "/" + file

    # Load the CSV file
    raw_data = pd.read_csv(file_path)

    # Get headers
    headers = raw_data.columns
    
    # Initialize the dictionary
    for header in headers:
        dict[header] = []

    return dict

In [26]:
def get_data(folder_path, algorithm):

    """
    Get the data from the CSV files in the folder
    """
    
    # Get the files in the folder
    algo_folder_path = folder_path + algorithm

    # Get all the CSV files in the folder
    files = [f for f in listdir(algo_folder_path) if isfile(join(algo_folder_path, f))]
    
    # Initialize the dictionary to store the data
    float_data = get_header_initialized_dict(files[0], algo_folder_path)

    # Convert the data to float
    for file in files:

        # Print the file path
        file_path = algo_folder_path + "/" + file

        # Load the CSV file
        raw_data = pd.read_csv(file_path)

        # Get headers
        headers = raw_data.columns

        raw_data_dict = {}
        for header in headers:
            raw_data_dict[header] = None

        # Store the data in the dictionary
        for header in headers:
            raw_data_dict[header] = raw_data[header].values

        for key in raw_data_dict:
            if key == 'Planner':
                continue
            float_data[key].extend(raw_data_dict[key].astype(float).tolist())
    
            # If there's any nan data - FASTER approach sometimes have nan values in Dynamic A* - remove them
            if (np.isnan(float_data[key]).any()):
                float_data[key] = [x for x in float_data[key] if not np.isnan(x)] 

    return float_data


In [27]:
def get_metrics(float_data):
    """
    Get the average, min, max, and std of the data
    """

    # Initialize the dictionary
    avg_data = {}
    min_data = {}
    max_data = {}
    std_data = {}

    # Get average, min, max, and std of the data
    for key in float_data:

        if key == 'Planner':
            continue

        avg_data[key] = np.mean(np.array(float_data[key]))
        min_data[key] = np.min(np.array(float_data[key]))
        max_data[key] = np.max(np.array(float_data[key]))
        std_data[key] = np.std(np.array(float_data[key]))

    return avg_data, min_data, max_data, std_data

In [ ]:
def filter_data(data):

    # Cost (final node's g): if global planning time is 0.0 and the result is 0, then it's after reaching the goal
    failure_indices = [i for i, x in enumerate(data["Global Planning Time [ms]"]) if x == 0.0 and data['Result'][i] == 0]

    print(failure_indices)

    # Filter out the failures
    
    # Get the keys
    keys = list(data.keys())
    for key in keys:
        data[key] = [x for i, x in enumerate(data[key]) if i not in failure_indices]

    return data

In [30]:
# Get the data
for algorithm in algorithms:

    float_data = get_data(folder_path, algorithm)
    float_data = filter_data(float_data)
    avg_data, min_data, max_data, std_data = get_metrics(float_data)

    for key in float_data:
        if key == 'Planner':
            print("*******************************************************************")
            print("Algorithm: ", algorithm)
            print("*******************************************************************")
            continue

        print(key, avg_data[key], min_data[key], max_data[key], std_data[key])

*******************************************************************
Algorithm:  faster_4
*******************************************************************
Result 0.21671954752379638 0.0 1.0 0.4120099334298596
Cost (final node's g) 10.215733528762588 6.06439 18.7153 2.457401247770152
Total replanning time [ms] 171.14618896399503 47.1542 666.214 47.458119730356316
Global Planning Time [ms] 4.202238515657332 0.433 152.764 3.2423732838158053
CVX Decomposition Time [ms] 27.675711684370256 13.713 93.998 11.251567808888272
Gurobi Time [ms] 128.4062135466961 11.193 711.412 58.837235751992104
Gurobi Try Counter 7.823010070354532 1.0 25.0 3.194826180250184
 Yaw Time [ms] 5.687413160435922 2.102 13.392 1.0520645171859793
Static JPS Time [ms] 3.0618904676507106 0.09 12.066 2.702958316253807
Check Path Time [ms] 1.0886763691543662 0.093 8.399 1.0496401468877894
Dynamic A* Time [ms] 0.020670943708609273 0.0 141.155 1.6610046456812564
Recover Path Time [ms] 2.759381898454746e-06 0.0 0.012 0.0001693

In [31]:
# font
import matplotlib.font_manager as font_manager
font = font_manager.FontProperties()
font.set_family('serif')
plt.rcParams.update({"text.usetex": True})
plt.rcParams["font.family"] = "Times New Roman"
font.set_size(40)

In [32]:
!pip install numpy rclpy rosbag2_py rosidl-runtime-py

import numpy as np
import rosbag2_py
import rclpy
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
from dynus_interfaces.msg import Goal

In [33]:
def read_ros2_bag(bag_path, topic_name, message_type):
    """
    Reads a ROS 2 bag file and extracts messages from a specified topic.

    Args:
        bag_path (str): Path to the ROS 2 bag file.
        topic_name (str): The name of the topic to extract data from.
        message_type (str): The message type as a string (e.g., "geometry_msgs/msg/Twist").

    Returns:
        list: A list of messages from the specified topic.
    """
    storage_options = rosbag2_py.StorageOptions(uri=bag_path, storage_id="sqlite3")
    converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format="cdr", output_serialization_format="cdr"
    )

    reader = rosbag2_py.SequentialReader()
    reader.open(storage_options, converter_options)

    all_topics = reader.get_all_topics_and_types()
    topic_names = [topic.name for topic in all_topics]

    topic_found = any(t == topic_name for t in topic_names)
    if not topic_found:
        print(f"Topic '{topic_name}' not found in the bag.")
        return []

    # Resolve the message type dynamically
    msg_type = get_message(message_type)

    data = []
    while reader.has_next():
        topic, serialized_msg, _ = reader.read_next()
        if topic == topic_name:
            msg = deserialize_message(serialized_msg, msg_type)
            data.append(msg)

    return data


In [34]:
def process_goal_data(messages):
    """
    Processes goal data to extract desired values and computes averages.

    Args:
        messages (list): List of ROS 2 messages.

    Returns:
        np.ndarray: Array of extracted values.
    """
    
    # Get the very first msg's time stamp
    start_time = messages[0].header.stamp.sec + messages[0].header.stamp.nanosec * 1e-9

    # Get the very last msg's time stamp
    end_time = messages[-1].header.stamp.sec + messages[-1].header.stamp.nanosec * 1e-9

    # Get the total time
    total_time = end_time - start_time

    # Extract goal positions
    positions = [[msg.p.x, msg.p.y, msg.p.z] for msg in messages]

    # Extract velocities
    velocities = [np.linalg.norm(np.array([msg.v.x, msg.v.y, msg.v.z])) for msg in messages]

    # Extract accelerations
    accelerations = [np.linalg.norm(np.array([msg.a.x, msg.a.y, msg.a.z])) for msg in messages]

    # Extract jerk
    jerks = [np.linalg.norm(np.array([msg.j.x, msg.j.y, msg.j.z])) for msg in messages]

    return np.array(positions), np.array(velocities), np.array(accelerations), np.array(jerks), total_time

In [35]:
def plot_velocity_histogram(velocities, bins=30, title="Velocity Profile", xlabel="Velocity (m/s)", ylabel="Frequency", output_file=None):
    """
    Plots a histogram of the velocity profile.

    Args:
        velocities (np.ndarray): Array of velocity values.
        bins (int): Number of bins for the histogram. Default is 30.
        title (str): Title of the plot. Default is "Velocity Profile".
        xlabel (str): Label for the x-axis. Default is "Velocity (m/s)".
        ylabel (str): Label for the y-axis. Default is "Frequency".
        output_file (str): If provided, saves the plot to this file. Default is None.
    """
    # Create the histogram
    plt.figure(figsize=(10, 6))
    weights = velocities / len(velocities)
    plt.hist(velocities, weights=weights, bins=bins, color="skyblue", edgecolor="black", alpha=0.7)
    
    # Add titles and labels
    plt.title(title, fontproperties=font, weight='bold')
    plt.xlabel(xlabel, fontproperties=font)
    plt.ylabel(ylabel, fontproperties=font)
    plt.xticks(fontproperties=font)
    plt.yticks(fontproperties=font)
    
    # Grid and styling
    plt.grid(color='gray', linewidth=0.5, alpha=0.7)
    plt.tight_layout()
    
    # Show or save the plot
    if output_file:
        plt.savefig(output_file, bbox_inches='tight', dpi=300)
        print(f"Plot saved as {output_file}")
    else:
        plt.show()

In [36]:
# Get FASTER_4's data 
base_path = "/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4"
num_bags = 10
topic_name = "/NX01/goal"
message_type = "dynus_interfaces/msg/Goal"  # Update with your message type

# Initialize arrays to store data
faster_4_pos_array = np.array([])
faster_4_vel_array = np.array([])
faster_4_accel_array = np.array([])
faster_4_jerk_array = np.array([])
faster_4_travel_time_array = np.array([])

for i in range(num_bags):
    bag_path = f"{base_path}/num_{i}/num_{i}"
    print(f"processing bag: {bag_path}")

    # Read the bag and extract data
    messages = read_ros2_bag(bag_path, topic_name, message_type)

    if not messages:
        print(f"No messages found or topic does not exist in {bag_path}.")
    else:
        
        # Process data
        pos, vel, accel, jerk, travel_time = process_goal_data(messages)

        # Append to the arrays
        faster_4_pos_array = np.append(faster_4_pos_array, pos)
        faster_4_vel_array = np.append(faster_4_vel_array, vel)
        faster_4_accel_array = np.append(faster_4_accel_array, accel)
        faster_4_jerk_array = np.append(faster_4_jerk_array, jerk)
        faster_4_travel_time_array = np.append(faster_4_travel_time_array, travel_time)

processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_0/num_0


[INFO] [1735753932.558110758] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_0/num_0/num_0_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_1/num_1


[INFO] [1735753933.321200632] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_1/num_1/num_1_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_2/num_2


...

[INFO] [1735753937.459048425] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_7/num_7/num_7_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_8/num_8


[INFO] [1735753938.124108710] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_8/num_8/num_8_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_9/num_9


[INFO] [1735753938.760137350] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_9/num_9/num_9_0.db3' for READ_ONLY.


In [37]:
# Get FASTER_5's data 
base_path = "/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5"
num_bags = 10
topic_name = "/NX01/goal"
message_type = "dynus_interfaces/msg/Goal"  # Update with your message type

# Initialize arrays to store data
faster_5_pos_array = np.array([])
faster_5_vel_array = np.array([])
faster_5_accel_array = np.array([])
faster_5_jerk_array = np.array([])
faster_5_travel_time_array = np.array([])

for i in range(num_bags):
    bag_path = f"{base_path}/num_{i}/num_{i}"
    print(f"processing bag: {bag_path}")

    # Read the bag and extract data
    messages = read_ros2_bag(bag_path, topic_name, message_type)

    if not messages:
        print(f"No messages found or topic does not exist in {bag_path}.")
    else:
        
        # Process data
        pos, vel, accel, jerk, travel_time = process_goal_data(messages)

        # Append to the arrays
        faster_5_pos_array = np.append(faster_5_pos_array, pos)
        faster_5_vel_array = np.append(faster_5_vel_array, vel)
        faster_5_accel_array = np.append(faster_5_accel_array, accel)
        faster_5_jerk_array = np.append(faster_5_jerk_array, jerk)
        faster_5_travel_time_array = np.append(faster_5_travel_time_array, travel_time)

processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_0/num_0


[INFO] [1735753939.407588820] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_0/num_0/num_0_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_1/num_1


[INFO] [1735753940.159242695] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_1/num_1/num_1_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_2/num_2


...

[INFO] [1735753944.185836168] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_7/num_7/num_7_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_8/num_8


[INFO] [1735753944.871999260] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_8/num_8/num_8_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_9/num_9


[INFO] [1735753945.528810310] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_9/num_9/num_9_0.db3' for READ_ONLY.


In [38]:
# Get FASTER_6's data 
base_path = "/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6"
num_bags = 10
topic_name = "/NX01/goal"
message_type = "dynus_interfaces/msg/Goal"  # Update with your message type

# Initialize arrays to store data
faster_6_pos_array = np.array([])
faster_6_vel_array = np.array([])
faster_6_accel_array = np.array([])
faster_6_jerk_array = np.array([])
faster_6_travel_time_array = np.array([])

for i in range(num_bags):
    bag_path = f"{base_path}/num_{i}/num_{i}"
    print(f"processing bag: {bag_path}")

    # Read the bag and extract data
    messages = read_ros2_bag(bag_path, topic_name, message_type)

    if not messages:
        print(f"No messages found or topic does not exist in {bag_path}.")
    else:
        
        # Process data
        pos, vel, accel, jerk, travel_time = process_goal_data(messages)

        # Append to the arrays
        faster_6_pos_array = np.append(faster_6_pos_array, pos)
        faster_6_vel_array = np.append(faster_6_vel_array, vel)
        faster_6_accel_array = np.append(faster_6_accel_array, accel)
        faster_6_jerk_array = np.append(faster_6_jerk_array, jerk)
        faster_6_travel_time_array = np.append(faster_6_travel_time_array, travel_time)

processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_0/num_0


[INFO] [1735753946.172479264] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_0/num_0/num_0_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_1/num_1


[INFO] [1735753946.880077654] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_1/num_1/num_1_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_2/num_2


...

[INFO] [1735753950.913051096] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_7/num_7/num_7_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_8/num_8


[INFO] [1735753951.633473537] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_8/num_8/num_8_0.db3' for READ_ONLY.


processing bag: /media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_9/num_9


[INFO] [1735753952.288845912] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_9/num_9/num_9_0.db3' for READ_ONLY.


In [39]:
import numpy as np
from scipy.spatial.distance import euclidean
import rosbag2_py
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message

In [40]:
def compute_path_length(positions):
    """
    Computes the total path length by summing distances between consecutive positions.

    Args:
        positions (np.ndarray): An array of position vectors [x, y, z].

    Returns:
        float: The total path length.
    """
    path_length = 0.0
    for i in range(1, len(positions)):
        path_length += euclidean(positions[i - 1], positions[i])
    return path_length

In [41]:
# Main processing loop (modification to calculate average travel time and path length)
base_path = "/media/kkondo/T7/dynus/tro_paper/time_allocation/bags"
num_bags = 10
topic_name = "/NX01/goal"
message_type = "dynus_interfaces/msg/Goal"  # Update with your message type
algorithms = ["liu", "faster_4", "faster_5", "faster_6"]

for algorithm in algorithms:

    # Initialize arrays to store data
    travel_time_list = []
    path_length_list = []

    for i in range(num_bags):
        bag_path = f"{base_path}/{algorithm}/num_{i}/num_{i}"
        # print(f"Processing bag: {bag_path}")

        # Read the bag and extract data
        messages = read_ros2_bag(bag_path, topic_name, message_type)

        if not messages:
            print(f"No messages found or topic does not exist in {bag_path}.")
            continue
        
        # Process data
        pos, vel, accel, jerk, travel_time = process_goal_data(messages)

        # Compute path length
        path_length = compute_path_length(pos)

        # Append travel time and path length
        travel_time_list.append(travel_time)
        path_length_list.append(path_length)

        # Print the results for each bag
        # print(f"Travel Time: {travel_time:.2f} seconds")
        # print(f"Path Length: {path_length:.2f} meters")

    # Compute averages
    average_travel_time = np.mean(travel_time_list)
    average_path_length = np.mean(path_length_list)

    print("*******************************************************************")
    print(f"Algorithm: {algorithm}")
    print(f"Average Travel Time: {average_travel_time:.2f} seconds")
    print(f"Average Path Length: {average_path_length:.2f} meters")
    print("*******************************************************************")

[INFO] [1735753952.973598552] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/liu/num_0/num_0/num_0_0.db3' for READ_ONLY.
[INFO] [1735753953.981413018] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/liu/num_1/num_1/num_1_0.db3' for READ_ONLY.
[INFO] [1735753954.809867905] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/liu/num_2/num_2/num_2_0.db3' for READ_ONLY.
[INFO] [1735753955.866252152] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/liu/num_3/num_3/num_3_0.db3' for READ_ONLY.
[INFO] [1735753956.851529387] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/liu/num_4/num_4/num_4_0.db3' for READ_ONLY.
[INFO] [1735753957.701728390] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/liu/num_5/num_5/num_5_0.db3' for READ_ONLY.
[INFO] [17357539

*******************************************************************
Algorithm: liu
Average Travel Time: 47.69 seconds
Average Path Length: 101.61 meters
*******************************************************************


[INFO] [1735753961.759722689] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_0/num_0/num_0_0.db3' for READ_ONLY.
[INFO] [1735753962.512131706] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_1/num_1/num_1_0.db3' for READ_ONLY.
[INFO] [1735753963.199960009] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_2/num_2/num_2_0.db3' for READ_ONLY.
[INFO] [1735753963.892644675] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_3/num_3/num_3_0.db3' for READ_ONLY.
[INFO] [1735753964.663166974] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_4/num_4/num_4_0.db3' for READ_ONLY.
[INFO] [1735753965.369598100] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_4/num_5/num_5/num_5_0.db3' f

*******************************************************************
Algorithm: faster_4
Average Travel Time: 36.38 seconds
Average Path Length: 102.12 meters
*******************************************************************


[INFO] [1735753968.853174057] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_0/num_0/num_0_0.db3' for READ_ONLY.
[INFO] [1735753969.607067346] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_1/num_1/num_1_0.db3' for READ_ONLY.
[INFO] [1735753970.283888912] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_2/num_2/num_2_0.db3' for READ_ONLY.
[INFO] [1735753970.960371184] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_3/num_3/num_3_0.db3' for READ_ONLY.
[INFO] [1735753971.656983711] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_4/num_4/num_4_0.db3' for READ_ONLY.
[INFO] [1735753972.331513084] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_5/num_5/num_5/num_5_0.db3' f

*******************************************************************
Algorithm: faster_5
Average Travel Time: 35.46 seconds
Average Path Length: 102.34 meters
*******************************************************************


[INFO] [1735753975.620087612] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_0/num_0/num_0_0.db3' for READ_ONLY.
[INFO] [1735753976.309780236] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_1/num_1/num_1_0.db3' for READ_ONLY.
[INFO] [1735753976.970340494] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_2/num_2/num_2_0.db3' for READ_ONLY.
[INFO] [1735753977.646109392] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_3/num_3/num_3_0.db3' for READ_ONLY.
[INFO] [1735753978.283933457] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_4/num_4/num_4_0.db3' for READ_ONLY.
[INFO] [1735753979.017241423] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/time_allocation/bags/faster_6/num_5/num_5/num_5_0.db3' f

*******************************************************************
Algorithm: faster_6
Average Travel Time: 35.08 seconds
Average Path Length: 102.53 meters
*******************************************************************
